<a href="https://colab.research.google.com/github/aidan-lew/junk-drawer/blob/main/finrl_fingpt_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are the recourses to make a venv for colab:
https://stackoverflow.com/questions/74244113/how-to-make-virtual-environment-on-google-colab-that-can-be-saved-on-drive
https://colab.research.google.com/drive/1Vg005uKhgt-ZwyQ11-BGceqY0q07qIzF

In [ ]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12

# Load FinGPT

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
#have to have access to LLama -  via hugging face
### THIS IS ABlE TO RUN ON a T4!!!
# https://github.com/AI4Finance-Foundation/FinGPT/tree/master/fingpt/FinGPT_Forecaster
# https://github.com/AI4Finance-Foundation/FinGPT/blob/master/FinGPT_Inference_Llama2_13B_falcon_7B_for_Beginners.ipynb
### see how the flow works for these, look at the hugging face page dataset for training.



#### note, this is for the forcaster FinGPT model, which is not specifically for sentiment analysis
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   # optional if you have enough VRAM
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
gpt_model = gpt_model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Whole pipeline

## Scraper

In [ ]:
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles


## Analyzer

In [ ]:
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      #print(sentiments)
  return sentiments

## Predictor

In [ ]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')
    print(stock_data)
    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    #print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']
    print(stock_data)
    #print(f"X shape: {X.shape}")
    #print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    #print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    #print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


## Pipeline Function

In [ ]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  #return ticker
  #return num_articles
  #return prediction

In [ ]:
prediction_pipeline()

tickerAAPL
number of articles10


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-06-26 09:30:00-04:00  211.449997  211.449997  211.449997  211.449997   
2024-06-26 09:31:00-04:00  211.429993  211.600006  210.889999  211.365005   
2024-06-26 09:32:00-04:00  211.354996  212.059998  211.100006  211.789597   
2024-06-26 09:33:00-04:00  211.759995  212.270004  211.679993  212.139999   
2024-06-26 09:34:00-04:00  212.139999  212.289993  211.979996  212.100006   
...                               ...         ...         ...         ...   
2024-06-26 14:47:00-04:00  214.386002  214.389008  214.369995  214.384995   
2024-06-26 14:48:00-04:00  214.389999  214.440002  214.374695  214.419998   
2024-06-26 14:49:00-04:00  214.429993  214.500000  214.419998  214.464996   
2024-06-26 14:50:00-04:00  214.460007  214.470001  214.370102  214.375000   
2024-06-26 14:51:00-04:00  214.320007  214.320007  214.320007  214.320007   

# Piece by piece

## Scraper

In [ ]:
####### News sources - what about econ data, general news, etc?
#### googlenews API (https://pypi.org/project/GoogleNews/)
### Rueters (https://www.reuters.com/business/)
### https://www.bloomberg.com/
from googlesearch import search
from bs4 import BeautifulSoup
import requests

def scrape_news_headlines(ticker, num_articles):
  query = f"{ticker} news" #####find a better url/website because the current one just gets the google web results
  search_results = search(query)
  articles = []
  count = 0
  for url in search_results:
      if count >= num_articles:
          break
      try:
          response = requests.get(url, verify=False, headers={'User-Agent':'Mozilla/5.0'}, #timeout=10
                                  )
          soup = BeautifulSoup(response.text, 'html.parser')
          title = soup.find('title').text if soup.find('title') else 'No title'
          articles.append({'title': title, 'link': url})
          count += 1
      except Exception as e:
          print(f"Error processing {url}: {e}")
  return articles

In [ ]:
ticker = 'AAPL'
num_articles = 5
articles = scrape_news_headlines(ticker, num_articles)
print(articles)
### returns a [{'':}] list of tuples

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[{'title': 'AAPL: Apple Inc - Stock Price, Quote and News - CNBC', 'link': 'https://www.cnbc.com/quotes/AAPL'}, {'title': 'No title', 'link': 'https://finance.yahoo.com/quote/AAPL/news/'}, {'title': 'Apple (AAPL) News Headlines | Nasdaq', 'link': 'https://www.nasdaq.com/market-activity/stocks/aapl/news-headlines'}, {'title': 'No title', 'link': 'https://sg.finance.yahoo.com/quote/AAPL/'}, {'title': 'marketwatch.com', 'link': 'https://www.marketwatch.com/investing/stock/aapl'}]


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.marketwatch.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Analyzer

In [ ]:
import torch
def analyze_news_sentiment(articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  gpt_model.to(device)  # Move the model to the device

  sentiments = []
  for article in articles:
      inputs = tokenizer(article['title'], return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      sentiments.append(sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      print(sentiments)
  return sentiments

In [ ]:
ticker = 'AAPL'
num_articles = 5
articles = scrape_news_articles(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)

NameError: name 'tokenizer' is not defined

In [ ]:
sentiments = analyze_news_sentiment(articles)

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

In [ ]:
sentiments

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
          1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
         [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
          1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
         [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
          2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
         ...,
         [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
          2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
         [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
          2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
         [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
          5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32),
 array([[[6.4145797e-04, 5.0152973e-03, 3.4871415e-04, ...,
          4.5422399e-03, 8.1572525e-02, 1.2902316e-03],
         [9.0460083e-04, 1.3417624e-04, 7.4513872e-07, ...,
          6.7129950e-03, 3.6121567e-03, 3.7188535e-03],
         [1.1063426e-02, 9.1692823e-04, 

## Predictor

In [ ]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def make_stock_price_prediction(ticker, sentiments):
    stock_data = yf.download(ticker, period='1d', interval='1m')

    # Extract the positive sentiment score (index 1) from each softmax output
    sentiment_scores = [sentiment[0, 1] for sentiment in sentiments]

    # Create a 1D numpy array from the sentiment scores
    sentiments = np.array(sentiment_scores)

    sentiment_mean = np.mean(sentiments)
    #print(f"Sentiment mean: {sentiment_mean}")

    stock_data['sentiment'] = [sentiment_mean] * len(stock_data)  # Create a new column with the same value repeated for each row
    X = stock_data[['sentiment']]
    y = stock_data['Close']
    print(stock_data)

    #print(f"X shape: {X.shape}")
    #print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [sentiment_mean]})
    #print(f"Prediction input shape: {prediction_input.shape}")

    prediction = fin_model.predict(prediction_input)

    print(f"Prediction: {prediction}")

    return prediction[0]  # Return the scalar prediction


TypeError: string indices must be integers

In [ ]:
make_stock_price_prediction(ticker, sentiments)

[*********************100%%**********************]  1 of 1 completed

Sentiment mean: 0.105434849858284
X shape: (99, 1)
y shape: (99,)
Prediction input shape: (1, 1)
Prediction: [209.95151697]


209.9515169702395

## Pipeline Function

In [ ]:
def prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  articles = scrape_news_headlines(ticker, num_articles)
  sentiments = analyze_news_sentiment(articles)
  prediction = make_stock_price_prediction(ticker, sentiments)
  print('\n' f"Predicted stock price for {ticker}: {prediction}")
  return ticker
  return num_articles
  return prediction

In [ ]:
prediction_pipeline()

tickerAAPL
number of articles10


NameError: name 'GoogleNews' is not defined

In [ ]:
ticker = 'AAPL'
num_articles = 10
prediction_pipeline(ticker, num_articles)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

209.94394170587714

In [ ]:
ticker = 'AAPL'
num_articles = 10

articles = scrape_news_headlines(ticker, num_articles)
sentiments = analyze_news_sentiment(articles)
prediction = make_stock_price_prediction(ticker, sentiments)

print(f"Predicted stock price for {ticker}: {prediction}")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32)]
[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, .

[*********************100%%**********************]  1 of 1 completed

[array([[[1.6988595e-03, 2.8256273e-02, 1.0302393e-03, ...,
         1.4731156e-02, 1.9216403e-01, 4.3249498e-03],
        [9.3285739e-03, 1.2565035e-01, 2.7878977e-07, ...,
         1.8767979e-02, 2.7468417e-02, 1.4430288e-03],
        [1.9861681e-03, 4.4266836e-04, 1.1267488e-05, ...,
         2.9526176e-02, 3.5546772e-02, 1.2635583e-03],
        ...,
        [8.7134868e-02, 8.1762232e-02, 1.5530892e-01, ...,
         2.5955172e-02, 3.3786733e-02, 3.1461842e-02],
        [4.7745969e-02, 5.5323094e-02, 2.8283855e-02, ...,
         2.4766596e-02, 9.8324986e-03, 2.0155214e-02],
        [5.4839859e-03, 9.5935218e-04, 1.8841110e-02, ...,
         5.7463129e-03, 1.7324112e-02, 5.2585866e-02]]], dtype=float32), array([[[1.56065449e-04, 2.30889302e-03, 7.26893340e-05, ...,
         1.96373928e-03, 9.80778132e-03, 2.92713230e-04],
        [2.20087561e-04, 6.15299141e-05, 1.56542086e-07, ...,
         2.90221884e-03, 4.34303569e-04, 8.43691756e-04],
        [3.48229078e-05, 5.06131255e-05, 1.0

# Usin Google News Instead of Search
Scraper & analyzer

In [ ]:
from GoogleNews import GoogleNews
#### changed name

def gn_scrape_news_articles(ticker, num_articles):
    gn = GoogleNews(lang='en')
    gn.search(ticker)
    gn_news_articles = gn.results(num_articles)
    gn_titles = [article['title'] for article in gn_news_articles]
    return gn_titles

#ticker = 'AAPL'
#num_articles = 5
#articles = scrape_news_articles(ticker, num_articles)
#print(articles)


### returns a [''] list

In [ ]:
import torch
def gn_analyze_news_sentiment(gn_articles):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  #gpt_model.to(device)  # Move the model to the device

  gn_sentiments = []
  for article in gn_articles:
      inputs = tokenizer(article, return_tensors='pt', max_length=512, truncation=True)
      inputs = {k: v.to(device) for k, v in inputs.items()}  # Move the input tensors to the device
      outputs = gpt_model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
      gn_sentiment = torch.nn.functional.softmax(outputs.logits, dim=1)
      gn_sentiments.append(gn_sentiment.detach().cpu().numpy())  # Move the sentiment tensor back to the CPU before converting to numpy
      #print(sentiments)
  return gn_sentiments

In [ ]:
#ticker = 'AAPL'
#num_articles = 5
#articles = scrape_news_articles(ticker, num_articles)
#sentiments = analyze_news_sentiment(articles)

In [ ]:
import numpy as np
import yfinance as yf
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd

def gn_make_stock_price_prediction(ticker, gn_sentiments):
    gn_stock_data = yf.download(ticker, period='1d', interval='1m')
    print(gn_stock_data)
    # Extract the positive sentiment score (index 1) from each softmax output
    gn_sentiment_scores = [sentiment[0, 1] for sentiment in gn_sentiments]

    # Create a 1D numpy array from the sentiment scores
    gn_sentiments = np.array(gn_sentiment_scores)

    gn_sentiment_mean = np.mean(gn_sentiments)
    #print(f"Sentiment mean: {sentiment_mean}")

    gn_stock_data['sentiment'] = [gn_sentiment_mean] * len(gn_stock_data)  # Create a new column with the same value repeated for each row
    X = gn_stock_data[['sentiment']]
    y = gn_stock_data['Close']
    print(gn_stock_data)
    gn_stock_data.head()
    #print(f"X shape: {X.shape}")
    #print(f"y shape: {y.shape}")

    # Fit a HistGradientBoostingRegressor model
    fin_model = HistGradientBoostingRegressor()
    fin_model.fit(X, y)

    # Make a prediction
    prediction_input = pd.DataFrame({'sentiment': [gn_sentiment_mean]})
    #print(f"Prediction input shape: {prediction_input.shape}")

    gn_prediction = fin_model.predict(prediction_input)

    #print(f"Prediction: {prediction}")

    return gn_prediction[0]  # Return the scalar prediction

ticker = 'AAPL'
num_articles = 5
gn_articles = scrape_news_articles(ticker, num_articles)
gn_sentiments = gn_analyze_news_sentiment(gn_articles)
gn_make_stock_price_prediction(ticker, gn_sentiments)

NameError: name 'scrape_news_articles' is not defined

In [ ]:
def gn_prediction_pipeline():
  ticker = input('ticker')
  num_articles = int(input('number of articles'))
  gn_articles = gn_scrape_news_articles(ticker, num_articles) #note the change in name
  gn_sentiments = gn_analyze_news_sentiment(gn_articles)
  gn_prediction = gn_make_stock_price_prediction(ticker, gn_sentiments)
  print('\n' f"Predicted stock price for {ticker}: {gn_prediction}")
  #return ticker
  #return num_articles
  #return prediction

In [ ]:
gn_prediction_pipeline()
#### gives the same prediction as using the search function,
### which means either that they are getting the same data, the softmax function is compressing things,
### and/or the regressor is just doing what it wants

tickerAAPL
number of articles10


[*********************100%%**********************]  1 of 1 completed


Predicted stock price for AAPL: 213.41759320601676


'AAPL'

# Notes on where I left off before being rudely interrupted: seeing if the sentiment scores are the same for either method

In [ ]:
gn_pred = gn_prediction_pipeline()
se_pred = prediction_pipeline()
### DEFINITELY giving same prediction despite mutually exclusive variables

tickerAAPL
number of articles10


[*********************100%%**********************]  1 of 1 completed



Predicted stock price for AAPL: 213.4197461528163
tickerAAPL
number of articles10


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cnbc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.yahoo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nasdaq.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:110


Predicted stock price for AAPL: 213.4197461528163


In [ ]:
gn_stock_data = yf.download(ticker, period='1d', interval='1m')

# Extract the positive sentiment score (index 1) from each softmax output
gn_sentiment_scores = [sentiment[0, 1] for sentiment in gn_sentiments]

# Create a 1D numpy array from the sentiment scores
gn_sentiments = np.array(gn_sentiment_scores)

gn_sentiment_mean = np.mean(gn_sentiments)
#print(f"Sentiment mean: {sentiment_mean}")

gn_stock_data['sentiment'] = [gn_sentiment_mean] * len(gn_stock_data)  # Create a new column with the same value repeated for each row
X = gn_stock_data[['sentiment']]
y = gn_stock_data['Close']


[*********************100%%**********************]  1 of 1 completed


NameError: name 'gn_sentiments' is not defined

# With Sentiment FinGPT
look at their hugging face (https://huggingface.co/FinGPT/fingpt-sentiment_llama2-13b_lora)


In [ ]:
!pip install transformers==4.32.0 peft==0.5.0
!pip install sentencepiece
!pip install accelerate
!pip install torch
!pip install peft
!pip install datasets
!pip install bitsandbytes



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
  Attempting uninstall: peft
    Found existing installation: peft 0.11.2.dev0
    Uninstalling peft-0.11.2.dev0:
      Successfully uninstalled peft-0.11.2.dev0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel  # 0.5.0
import torch
import os

### Might be too big for the T4 - 13B instead of 7B
base_model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-13b-hf",
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16, # optional if you have enough VRAM
    #offload_folder=offload_folder,
    )
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-13b-hf")

gpt_model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-sentiment_llama2-13b_lora')
gpt_model = gpt_model.eval()




# Make prompts
prompt = [
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is aggressively pursuing its growth strategy by increasingly focusing on technologically more demanding HDI printed circuit boards PCBs .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Answer: ''',
'''Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}
Input: A tinyurl link takes users to a scamming site promising that users can earn thousands of dollars by becoming a Google ( NASDAQ : GOOG ) Cash advertiser .
Answer: ''',
]

# Generate results
tokens = tokenizer(prompt, return_tensors='pt', padding=True, max_length=512)
res = gpt_model.generate(**tokens, max_length=512)
res_sentences = [tokenizer.decode(i) for i in res]
out_text = [o.split("Answer: ")[1] for o in res_sentences]

# show results
for sentiment in out_text:
    print(sentiment)

# Output:
# positive
# neutral
# negative


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1389: UserWarning: Current model requires 48237440 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: We need an `offload_dir` to dispatch this model according to this `device_map`, the following submodules need to be offloaded: base_model.model.model.layers.19, base_model.model.model.layers.20, base_model.model.model.layers.21, base_model.model.model.layers.22, base_model.model.model.layers.23, base_model.model.model.layers.24, base_model.model.model.layers.25, base_model.model.model.layers.26, base_model.model.model.layers.27, base_model.model.model.layers.28, base_model.model.model.layers.29, base_model.model.model.layers.30, base_model.model.model.layers.31, base_model.model.model.layers.32, base_model.model.model.layers.33, base_model.model.model.layers.34, base_model.model.model.layers.35, base_model.model.model.layers.36, base_model.model.model.layers.37, base_model.model.model.layers.38, base_model.model.model.layers.39, base_model.model.model.norm, base_model.model.lm_head.

# New Section